In [2]:
import copy
import math
import random
import networkx as nx
import xml.etree.ElementTree as ET
import xml.dom.minidom
from collections import namedtuple
from natsort import natsorted
from itertools import islice
import statistics
import pydotplus as dot

N = 8

random.seed(19225)

##########################################################################
class Packing(object):
    """Describes packing.

    Parameters
    ----------
    None

    Attributes
    ----------
    packing : Dict[str, ET.Element]
        A dictionary mapping the primitive output signals to top-level blocks.
    inputs : List[str]
        Primary inputs.
    outputs : List[str]
        Primary outputs.
    clocks : List[str]
        Primary clocks.
    tree : ET
        Parsed XML tree.
    root : ET.Element
        Root of the parsed tree.
    """
   
    #------------------------------------------------------------------------#
    def __init__(self):
        """Constructor of the Packing class."""

        self.packing = None
        self.tree = None
        self.root = None

        self.inputs = None
        self.outputs = None
        self.clocks = None
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def read_packing(self, filename):
        """Reads the packing in the form of a dictionary mapping the output
        signals of the primitives to the top-level blocks that contain them.
    
        Parameters
        ----------
        filename : str
            Name of the packed netlist file.
        
        Returns
        -------
        None
        """
    
        tree = ET.parse(filename)
        root = tree.getroot()
    
        packing = {}
        for block in root.findall("block"):
            for u in block.iter():
                if u.tag == "block" and u.attrib.get("instance").startswith("clb["):
                    packing.update({u.attrib["name"] : self.find_primitives(u)})
                    #print(u.attrib["name"],packing[u.attrib["name"]])
                    #print('**************************')
        self.packing = packing
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def find_primitives(self, block):
        """Finds all primitive (leaf) blocks.

        Parameters
        ----------
        block : ET.Element
            Block in which to seek the primitives.

        Returns
        -------
        Dict[str, ET.Element]
            Primitives within the block.
        """

        primitives = {}
        for node in block.iter():
            if node.tag == "block" and not node.findall("block"):
                name = node.attrib["name"]
                if name != "open":
                    primitives.update({name : node})

        return primitives
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def filter_underutilized_clusters(self, threshold = 0.8 * N):
        """Removes the underutilized clusters from the packing.

        Parameters
        ----------
        threshold : Optional[float], default = 0.8N
            Filling threshold below which the clusters are discarded.

        Returns
        -------
        None
        """

        rm_list = []
        for cluster, content in self.packing.items():
            if len(content) < threshold:
                rm_list.append(cluster)

        for cluster in rm_list:
            del self.packing[cluster]
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def induce_subgraphs(self, circuit_graph):
        """Extracts the subgraphs of the circuit graph induced by each of the
        different clusters.

        Parameters
        ----------
        circuit_graph : nx.DiGraph
            Circuit graph.

        Returns
        -------
        None
        """

        #Significant edges do not include LUT->FF ones.
        count_significant_edges = lambda subgraph : len([(u, v) for (u, v) in subgraph.edges()\
                                                         if subgraph.nodes[u].get("node_type", None) != "lut"\
                                                         or subgraph.nodes[v].get("node_type", None) != "ff"])

        #Compute the minimum number of significant edges for the cluster to be relevant.
        N_min = 0.8 * N
        K_avg = 4.2
        I = N_min * K_avg
        p_max = 0.8
        I_ext = K_avg * (N_min ** p_max)
        E = I - I_ext

        self.subgraphs = {}
        for cluster, content in self.packing.items():
            subgraph = circuit_graph.subgraph(content).copy()
            rm_list = []
            for u in subgraph.nodes():
                if 0 == subgraph.in_degree(u) == subgraph.out_degree[u]:
                    rm_list.append(u)
            for u in rm_list:
                subgraph.remove_node(u)
            if count_significant_edges(subgraph) > E:
                self.subgraphs.update({cluster : subgraph})
                #print(cluster," : ", subgraph)
                #print("*********")
    #------------------------------------------------------------------------#            

    #------------------------------------------------------------------------#            
    def export_graph(self):
        """Exports a compound subgraph, containing all packed and annealed clusters.

        Parameters
        ----------
        None

        Returns
        -------
        nx.DiGraph
            Compound graph.
        """

        G = nx.DiGraph()
        for c, clb in enumerate(self.placement):
            prefix = "clb%d_lut" % c
            for node, pos in self.placement[clb].items():
                G.add_node("%s%d" % (prefix, pos))
            for u, v in self.subgraphs[clb].edges():
                G.add_edge("%s%d" % (prefix, self.placement[clb][u]), "%s%d" % (prefix, self.placement[clb][v]))

        return G
    #------------------------------------------------------------------------#            
##########################################################################

##########################################################################
class Blif(object):
    """Models a circuit.

    Parameters
    ----------
    filename : str
        Name of the blif file.

    Attributes
    ----------
    G : nx.DiGraph
        Circuit graph
    """

    #------------------------------------------------------------------------#
    def __init__(self, filename):
        """Constructor of the Blif class."""

        self.G = nx.DiGraph()

        with open(filename, "r") as inf:
            self.txt = ''.join([line for line in inf.readlines() if line and not line.isspace()])
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def parse_lut(self, txt):
        """Parses the LUT text.

        Parameters
        ----------
        txt : str
            LUT text.

        Returns
        -------
        None
        """

        inputs = []
        output = None
        truth_table = ""
        rd = False
        for line in txt.splitlines():
            if not line or line.isspace():
                continue
            line = line.lstrip()
            if line.startswith('.'):
                if output is not None:
                    print ("LUT text contains multiple declarations.")
                    raise ValueError
                if not line.startswith(".names "):
                    print ("Instance not a LUT.")
                    raise ValueError
                rd = True
                words = line.split()
                if words[-1] == '\\':
                    words.pop(-1)
                inputs += words[1:]
                continue
            if rd:
                if any(line.startswith(x) for x in ('-', '0', '1')):
                    if output is None:
                        output = inputs.pop(-1)
                    truth_table += line if not truth_table else "\n%s" % line
                    continue
                words = line.split()
                if words[-1] == '\\':
                    words.pop(-1)
                inputs += words
            
        if output is None:
            if not inputs:
                print ("LUT declaration not found.")
                raise ValueError
            output = inputs.pop(-1)
        return output, inputs
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def parse_ff(self, txt):
        """Parses the FF instance.

        Parameters
        ----------
        txt : str
            FF text.

        Returns
        -------
        None
        """

        lines = txt.splitlines()
        if len(lines) != 1:
            print ("FF instance should have just one line of text.")
            raise ValueError

        line = lines[0].lstrip()
        if not line.startswith(".latch "):
            print("Not an FF instance.")
            raise ValueError

        words = line.split()[1:]
        if len(words) != 5:
            print("Parser expects a complete FF specification,"\
                  " including the clock signal, the trigger type,"\
                  " and the initial value.")
            raise ValueError

        d = words[0]
        q = words[1]
        edge_type = words[2]
        if edge_type not in ("fe", "re", "ah", "al", "as"):
            print("Invalid trigger type.")
            raise ValueError
        clk = words[3]
        init_val = words[4]
        if init_val not in ('0', '1', '2', '3'):
            print("Illegal initial value of the FF")
            raise ValueError

        return q, d
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def split_text(self):
        """Splits the text into . declarations.
        
        Parameters
        ----------
        None

        Returns
        -------
        List[str]
            Split declarations.
        """

        blocks = ['.' + block for block in self.txt.split("\n.") if block.split()[0] in ("names", "latch")]

        return blocks
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def build_graph(self):
        """Constructs the circuit graph.

        Parameters
        ----------
        None

        Returns
        -------
        None
        """

        lut_cnt = 0
        ff_cnt = 0

        for block in self.split_text():
            if block.startswith(".names"):
                output, inputs = self.parse_lut(block)
                #print('salut mec')
                #print(output)
                self.G.add_node(output, node_type = "lut")
                #print(self.G.nodes[output])
                for i in inputs:
                    self.G.add_edge(i, output)
                    #print(self.G.nodes[i])
            elif block.startswith(".latch"):
                q, d = self.parse_ff(block)
                self.G.add_node(q, node_type = "ff")
                self.G.add_edge(d, q)
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def pack_bles(self):
        """Packs BLEs together.

        Parameters
        ----------
        None

        Returns
        -------
        None
        """

        rm_list = []
        for u, attrs in self.G.nodes(data = True):
            if attrs.get("node_type", "ip") != "ff":
                continue
            p = list(self.G.pred[u])[0]
            if self.G.out_degree(p) == 1:
                for pp in self.G.pred[p]:
                    self.G.add_edge(pp, u)
                rm_list.append(p)

        for u in rm_list:
            self.G.remove_node(u)
    #------------------------------------------------------------------------#

    #------------------------------------------------------------------------#
    def sweep(self):
        """Removes the buffers and inverters from the graph.

        Parameters
        ----------
        None
        
        Returns
        -------
        None
        """

        updated = True
        while updated:
            rm_list = []
            updated = False
            for u, attrs in self.G.nodes(data = True):
                if attrs.get("node_type", "subckt") == "lut" and self.G.in_degree(u) == 1:
                    rm_list.append(u)
                    updated = True
                    p = list(self.G.pred[u])[0]
                    for v in self.G[u]:
                        self.G.add_edge(p, v)
            for u in rm_list:
                self.G.remove_node(u)
    #------------------------------------------------------------------------#
##########################################################################

##########################################################################
def anneal_placement(graph, objective = "wl"):
    """Performs a quick anneal of the LUT placement.

    Parameters
    ----------
    graph : nx.DiGraph
        Connectivity between the LUTs to position.
    objective : Optional[str or List[Tuple[nx.DiGrph, Dict[str, int]]], default = wl
        Optimization objective; wirelength or similarity with the given placements.

    Returns
    -------
    Dict[str, int]
        Annealed placement.
    """

    #------------------------------------------------------------------------#
    def evaluate(placement):
        """Evaluates the optimization objective

        Parameters
        ----------
        placement
            Placement.

        Returns
        -------
        float
            Objective.
        """

        obj = 0
        if objective == "wl":
            for u, v in graph.edges():
                obj += abs(placement[v] - placement[u])

            return obj

        for comp_graph, comp_placement in objective:
            local_obj = 0
            inverse_comp_placement = {p : u for u, p in comp_placement.items()}
            for u, v in graph.edges():
                if comp_graph.has_edge(inverse_comp_placement.get(placement[u], -1), inverse_comp_placement.get(placement[v], -1)):
                    local_obj += 1
            obj += local_obj

        obj *= 1.0 / len(objective)

        return -1 * obj #We perform minimization, but want to maximize the edge overlaps.
    #------------------------------------------------------------------------#

    nodes = list(graph.nodes())
    random.shuffle(nodes)
    init_placement = {u : i for i, u in enumerate(nodes)}

    #print(init_placement)  ***********

    init_obj = evaluate(init_placement)


    init_temp_comp_moves = 1000
    deltas = []
    for i in range(0, init_temp_comp_moves):
        swap_a = random.choice(list(init_placement.keys()))
        swap_b = swap_a
        while swap_b == swap_a:
            swap_b = random.choice(list(init_placement.keys()))
        orig_a = init_placement[swap_a]
        orig_b = init_placement[swap_b]
        init_placement[swap_a] = orig_b
        init_placement[swap_b] = orig_a
        new_obj = evaluate(init_placement)
        deltas.append(new_obj - init_obj)
        init_placement[swap_a] = orig_a
        init_placement[swap_b] = orig_b

    temperature = 2 * statistics.stdev(deltas)
    #print(temperature, init_obj) *****************

    best_obj = init_obj
    best_placement = copy.deepcopy(init_placement)

    placement = init_placement

    move_limit = int(math.ceil(10 * graph.number_of_nodes() ** (4.0 / 3)))
    obj = init_obj
    while temperature > (0.005 * obj * (1 if objective == "wl" else -1)) / graph.number_of_nodes():
        proposed = 0
        accepted = 0
        for i in range(0, move_limit):
            swap_a = random.choice(list(placement.keys()))
            swap_b = swap_a
            while swap_b == swap_a:
                swap_b = random.choice(list(placement.keys()))
            proposed += 1
            orig_a = placement[swap_a]
            orig_b = placement[swap_b]
            placement[swap_a] = orig_b
            placement[swap_b] = orig_a
            new_obj = evaluate(placement)
            if new_obj < obj:
                obj = new_obj
                accepted += 1
                if obj <= best_obj:
                    best_obj = obj
                    best_placement = copy.deepcopy(placement)            
            else:
                energy = math.exp(-1 * float(new_obj - obj) / temperature)
                toss = random.random()
                if toss < energy:
                    obj = new_obj
                    accepted += 1
                    continue
                placement[swap_a] = orig_a
                placement[swap_b] = orig_b

        alpha = float(accepted) / proposed
        temp_scale_fac = 1.0
        if alpha > 0.96:
            temp_scale_fac = 0.5
        elif alpha > 0.8:
            temp_scale_fac = 0.9
        elif alpha > 0.15:
            temp_scale_fac = 0.95
        else:
            temp_scale_fac = 0.8
        temperature *= temp_scale_fac

        #print(temperature, obj)

    #print("Annealed", best_obj) ***************************

    return best_placement
##########################################################################

circ = "bgm"

blif = Blif("vpr_runs/benchmarks/%s.blif" % circ)
blif.build_graph()
print(blif.G.number_of_edges())
blif.sweep()
print(blif.G.number_of_edges())
blif.pack_bles()
print(blif.G.number_of_edges())

packing = Packing()
packing.read_packing("vpr_runs/packing_%d/%s.net" % (N, circ))
packing.filter_underutilized_clusters()

packing.induce_subgraphs(blif.G)
print(len(packing.subgraphs), len(packing.packing))
ANCHOR_CNT = 16
#Clusters optimized for minimal WL, that are later used to maximize similarity.

clbs = list(packing.subgraphs)
random.shuffle(clbs)

packing.placement = {}
obj = []
for net, graph in packing.subgraphs.items():
    if len(packing.placement) < ANCHOR_CNT:
        placement = anneal_placement(graph, objective = "wl")
        packing.placement.update({net : placement})
        #print(packing.placement)
        #print("*****************")
        obj.append((graph, placement))
    else:
        placement = anneal_placement(graph, objective = obj)
        packing.placement.update({net : placement})

edgess = packing.export_graph().edges()



167613
163572
159373
238 3925


In [ ]:
edges = list(edgess)
print(edges)
edges = edges[:497]


In [4]:
def inputs_generator(number_lvls, multiplexer_perlvl , edges, max_jump):
    """
    Parameters
    ----------
    number_lvls : int
        the number multiplexer levels seperating outputs from inputs
    multiplexer_perlvl : dictionary
        a mapping from ( level(int) : number_ofMultiplexers(int) )
    edges : list[(str,str)]
        a list of couples of nodes representing the routings we want to make
        in the graph
    max_jump : int
        the maximum number of levels between 2 nodes an edge can cross
    Returns
    -------
    (int, dictionary, nx.DiGraph, List[str], int)
    The int being the total number of different clbs.
    The dictionary being a mapping (source : List[sinks])
    The Graph, is the one where we will route the different
        sources to their sinks that we have in the dictionary
    The List of strings is a list of all the switchTypes
    The last int is the number of luts
  
    """

    #*****************************************************************************************
    # This part of the code computes the number of different clbs, the number of luts per clb and computs also
    # a dictionary that maps sources to their sinks. all of this is computed from the edges input.
    
    i = '_input'
    o = '_output'
    src_sinks={}
    number_luts = 0
    number_clbs =0
    for (u,v) in edges :
        ind_1= u.find("b") + 1
        ind_2= u.find("_")
        s= u[ind_1:ind_2] # s is the clb number (extracted from the string)
        
        if int(s) > number_clbs :
            number_clbs = int(s)
        
        if (u+o) in src_sinks :
            src_sinks[(u+o)].append((v+i))
        else :
            src_sinks[(u+o)]=[(v+i)]
        
        ind= u.find("t") + 1
        lut_num = int(u[ind:]) #lut_num is the lut number (extracted from the string)
        if lut_num > number_luts :
            number_luts = lut_num
        ind= v.find("t") + 1
        lut_num = int(v[ind:])
        if lut_num > number_luts :
            number_luts = lut_num
            
    number_luts = number_luts +1
    number_clbs = number_clbs +1
    #*****************************************************************************************
    print( "number_luts : " ,number_luts)
    graph = make_graph(number_luts, number_clbs, number_lvls, multiplexer_perlvl, max_jump)
    
    return (number_clbs, src_sinks, graph[0],graph[1], number_luts)

def make_graph(number_luts, number_clbs, number_lvls, multiplexer_perlvl, max_jump):
    """
    This function constructs a graph.
    In this version, only the muxs in the last level have an edge to the inputs.
        
    Parameters
    ----------
    number_luts : int
        number of luts
    number_clbs : int
        number of differents clbs
    number_lvls : int
        number of lvls of multiplexers
    multiplexer_perlvl : dictionary
        a mapping from ( level(int) : number_ofMultiplexers(int) )
    max_jump : int
        the maximum number of levels between 2 nodes an edge can cross

    Returns
    -------
    (nx.DiGraph, list[str])
    
    The list of strings contains all switchTypes.
        
    """
    
    i = '_input'
    o = '_output'
    graph = nx.DiGraph()
    listOf_switchTypes= set()
    j=0
    for num_clb in range(number_clbs):
        j=0
        list_toLinkto =[] 
        listOf_currentOutputs =[]
        listOf_currentInputs =[] #version where only last multiplixers are connected to the input
        for num_lut in range(number_luts) :
            
            s = 'clb'+str(num_clb)+'_lut'+str(num_lut)
            graph.add_node((s+i),TypeOfNode = 'lut')
            graph.add_node((s+o),TypeOfNode = 'lut')
            graph.nodes[(s+i)]['clb'] = num_clb
            graph.nodes[(s+o)]['clb'] = num_clb
            graph.nodes[(s+o)]['TypeOfSwitch'] = 'lut'+str(num_lut)+o
            graph.nodes[(s+i)]['TypeOfSwitch'] = 'lut'+str(num_lut)+i
            graph.nodes[(s+i)]['lvl'] = number_lvls+1 
            graph.nodes[(s+o)]['lvl'] = 0
            graph.nodes[(s+i)]['p'] = 0
            graph.nodes[(s+o)]['p'] = 0
            graph.nodes[(s+i)]['h'] = 0
            graph.nodes[(s+o)]['h'] = 0
            graph.nodes[(s+i)]['isZero'] = False
            graph.nodes[(s+o)]['isZero'] = False
            for r in range(6):
                t = s + '_port' + str(r)
                graph.add_node(t , TypeOfNode ='port')
                graph.nodes[t]['clb'] = num_clb
                graph.nodes[t]['TypeOfSwitch'] = 'lut'+str(num_lut)+'_port'+str(r)
                graph.nodes[t]['lvl'] = number_lvls+1
                graph.nodes[t]['p'] = 0
                graph.nodes[t]['h'] = 0
                graph.nodes[t]['isZero'] = False
                graph.add_edge(t, (s+i))
                list_toLinkto.append(t) 
                
            listOf_currentOutputs.append((s+o))
            
        
        for num_lvl in range(number_lvls,0,-1) :
            list_toLinkBis = []
            number_muxs = multiplexer_perlvl[num_lvl]
            for num_mux in range(number_muxs) :
                s_name ='clb'+str(num_clb)+'_level'+str(num_lvl)+'_mux'+str(num_mux)
                s_type ='level'+str(num_lvl)+'_mux'+str(num_mux)
                graph.add_node(s_name,TypeOfNode = 'multiplexer')
                graph.nodes[s_name]['clb'] = num_clb
                graph.nodes[s_name]['TypeOfSwitch'] = s_type
                graph.nodes[s_name]['lvl']= num_lvl
                graph.nodes[s_name]['p'] = 0
                graph.nodes[s_name]['h'] = 0
                graph.nodes[s_name]['isZero'] = False
                graph.nodes[s_name]['jump'] = 1
                list_toLinkBis.append(s_name)
                listOf_switchTypes.add((s_type,1))  #we can do this for only the first clb
                for node in list_toLinkto :
                    graph.add_edge(s_name, node)
                    
            if num_lvl == (number_lvls):  #version where only last multiplixers are connected to the input
                list_toLinkto = []
            elif j > max_jump:
                list_toLinkto = list_toLinkto[multiplexer_perlvl[num_lvl+j]:]
                j= j-1
                
            list_toLinkto = list_toLinkto + list_toLinkBis
            j=j+1
        
        for output in listOf_currentOutputs :
            for node in list_toLinkto :
                graph.add_edge(output,node)
                
    # edge splitters
    listOf_edgesToAdd =[]
    listOf_nodesToAdd =[]
    listOf_edgesToRemove=[]
    
    for (u,v) in graph.edges() :
        jump = graph.nodes[v].get('lvl') - graph.nodes[u].get('lvl')
        if jump <= max_jump :
            if graph.nodes[v].get('TypeOfNode') == 'lut':
                s_name = (u + ' -> '+ v)
            
            else :  
                s_name = (u + ' -> '+ v)
                s_type = graph.nodes[u].get('TypeOfSwitch') + ' -> ' + graph.nodes[v].get('TypeOfSwitch')
                clb = graph.nodes[u].get('clb')
                listOf_nodesToAdd.append((s_name,s_type,jump,clb))
                listOf_edgesToAdd.append((u,s_name))
                listOf_edgesToAdd.append((s_name,v))
                listOf_switchTypes.add((s_type,jump)) #we can do this for only the first clb
                listOf_edgesToRemove.append((u,v))
        else :
            listOf_edgesToRemove.append((u,v))
        
       
    for node in listOf_nodesToAdd :
        graph.add_node(node[0], TypeOfNode = 'edge_splitter')
        graph.nodes[node[0]]['clb'] = node[3]
        graph.nodes[node[0]]['TypeOfSwitch'] = node[1]
        graph.nodes[node[0]]['jump'] = node[2]
        graph.nodes[node[0]]['p'] = 0
        graph.nodes[node[0]]['h'] = 0
        graph.nodes[node[0]]['isZero'] = False
        
    graph.remove_edges_from(listOf_edgesToRemove)
    graph.add_edges_from(listOf_edgesToAdd)
        
    
    return (graph, listOf_switchTypes)

In [5]:
number_lvls = 2
a=10
multiplexer_perlvl ={1 : a, 2: a}
max_jump = 1
outputs = inputs_generator(number_lvls, multiplexer_perlvl, edges, max_jump)


number_luts :  8


In [9]:

num_ports = 6
def routing(theGraph, clbNumber, nets, switchTypesList,num_luts ,num_muxs, numLvls, maxjump,opt=True, t = 10 **(-12), b=10 **(-12), s= 10**(-9),delta = 0.1, sc=10**(-12), max_crit=0.99, beta=8, crit=0.2, iter_to_zero =25, teta =1.1, firstIteration_avalancheCost =  1e-12):
    """
      
    Parameters 
    ---------
    theGraph : nx.Digraph
         a directed graph 
    clbNumber : int
         number of different clbs 
    nets : Dict
         a mapping of (source : [sinks]), [sinks] is basicly a list of strings
    switchTypes : List[str]
         a list of all switch types
    num_luts : int
        number of luts
    num_muxs : int
        the number of multiplexers per lvl
    numLvls : int
        number of lvls
    maxjump : int
        max jump permitted in the graph
    t : positive double
         timing cost of nodes
    b : positive double
         Fixed base cost of nodes
    s  : positive double
         starting cost of switches
    sc : positive double
         parameter determining the perceived cost of
         a potentiel switch when routing the most 
         critical possible net
    max_crit : double
         criticality
    beta : int
         criticality exponent
    crit : double (0 < crit < 1)
         slack ratio
    iter_to_zero : positive int
    teta : double
         adoption threshold
    firstIteration_avalancheCost : doublr
         the cost used in the fist iteration
         before ap and ah are computed.
         
    ---------
    
    returns
    ---------
    List[str]
        list of switch types chosen ( switch types here means multiplexers and edge splitters)
        
    """
     # Initialisation ------------------------------------------------------------------------------------
    
    
    graph = theGraph.copy()
    switchParam ={} # a dictionary where we will store the usage of each switch type.
    alreadyUpdatedUsage ={} #track in each iteration whether a switch type's usage has been updated by a
                            #particular clb number
    
    for st in switchTypesList :
        switchParam[st[0]]=[0,0,st[1]] #(U,UH,jump)
        alreadyUpdatedUsage[st[0]]={}
        for i in range(clbNumber):
            alreadyUpdatedUsage[st[0]][i]=False
    
    ap = 0# Parameter used to compute the avalanche cost, and that will be computed during the first iteration
    ah = 0# Parameter used to compute the avalanche cost, and that will be computed during the first iteration
    
    congestion_multiplier =0.5
    congestion_updater = 1.3
    
    selected_switches =[] #the return variable
    shared_resources = False
    iteration = 0 # Big iteration
    i=0 # small iteration
    
    #Auxilary methods-------------------------------------------------------------------------------------------
    
    def setNodesToZero(nodes) :
        """
        takes a list of nodes and puts their
        weight to zero.
        
        Parameters
        ----------
        nodes : the list of nodes whom we 
        want to ignore the weight
        Returns
        -------
        None
        """
        for node in nodes:
            graph.nodes[node]['isZero'] = True
            
    def weight(u, v, d) :
        """
        a method to be used as a parameter for 
        the nx.dijkstra_path function
        
        Parameters
        ----------
        u : node in the graph
        v : node in the graph
        d : attributes of the edge (u,v)
        
        Returns
        -------
        double
        
        """
        v_weight = 0
        if not(graph.nodes[v].get('isZero')) :
            node_type = graph.nodes[v].get('TypeOfNode')
            if (node_type == 'multiplexer' or node_type == 'multiplexer_noUsage' or node_type == 'port') :
                v_weight = crit * t + (1-crit)*b *congestion_multiplier *(graph.nodes[v].get('p'))*(graph.nodes[v].get('h'))
            if (node_type == 'multiplexer' or node_type =='edge_splitter') :    
                if (iteration == 1 and i ==1):
                    v_weight = v_weight + firstIteration_avalancheCost 
                else :
                    switchType = graph.nodes[v].get('TypeOfSwitch')
                    j = graph.nodes[v].get('jump')
                    v_weight = v_weight+avalancheCost_delay(switchParam[switchType][0],switchParam[switchType][1],j)

        return v_weight
    
    def avalancheCost(u, uh, j):
        """
        Parameters
        ----------
        u : double
            usage
        uh: double
            history usage
        j: integer
            jump 
        
        Returns
        -------
        double
        """
        return max([0,j*s*(1+delta)- ap*u - ah*uh])
    
    def avalancheCost_delay(u, uh, j): 
        """
        Parameters
        ----------
        u : double
            usage
        uh: double
            history usage
        j : int
            jump
        
        Returns
        -------
        double
        """
        return t + math.exp(math.log(sc/s)*((crit/max_crit)**beta)) * ( avalancheCost(u,uh,j) )
    
    def update_costs(path) :
        """
        updates the usage and the congestion
        of the nodes in the path list

        Parameters
        ----------
        path : set( str ) 
            a set of nodes from the graph
        
        Returns
        -------
        None

        """
        for node in path :
            n_type = graph.nodes[node].get('TypeOfNode')
            if (n_type == 'multiplexer' or n_type == 'edge_splitter') :
                switchtype = graph.nodes[node].get('TypeOfSwitch')
                clb = graph.nodes[node].get('clb')
                if ( not( alreadyUpdatedUsage[switchtype][clb]) ):
                    switchParam[switchtype][0]+=1
                    alreadyUpdatedUsage[switchtype][clb] = True
            
            if not(n_type == 'edge_splitter' or n_type == 'lut') :
                graph.nodes[node]['p'] +=1
        
    def updateHistoryCosts_andReset(i):
        """
        updates the history costs for the next iteration
        and resets to zero the usage and congestion for 
        all switches and wires
        
        Parameters
        ----------
        None
        
        Returns
        -------
        None
        """
        
        shared=False  
        for node in graph :
            p = graph.nodes[node].get('p')
            p = max([0,(p-1)])
            if p > 0 :
                shared=True
                if ( i== 150 or i == 151 ) :
                    print('probelm : ', node, 'p: ', graph.nodes[node].get('p'))
                    
                
        for st in switchParam:          
            for i in range(clbNumber):
                alreadyUpdatedUsage[st][i]= False
                       
        if shared :
            for node in graph :
                p = graph.nodes[node].get('p')
                h = graph.nodes[node].get('h')
                p = max([0,(p-1)])
                graph.nodes[node]['h']= h + func1(p)
                graph.nodes[node]['p']=0

            for sp in switchParam :
                u = max([0,(switchParam[sp][0]-1)]) 
                switchParam[sp][1] = switchParam[sp][1] + func2(u) 
                switchParam[sp][0] = 0
                
        return shared
    
    def func1(p):
        return p  # Can be set to more complex functions
    
    def func2(u):
        return u # Can be set to more complex functions
                
    def reset():
        """
        puts alreadyUpdatedUsage entries all to False
        and resets to zero congestion, usage and congestion
        history ( note : doesn't reset the usage history).
        
        Parameters
        ----------
        None
        
        Returns
        -------
        None
        """
        for st in switchParam:          
            for i in range(clbNumber):
                alreadyUpdatedUsage[st][i]= False
                
        for node in graph :
            graph.nodes[node]['h']=0
            graph.nodes[node]['p']=0
        
        for sp in switchParam :
            switchParam[sp][0] = 0
            switchParam[sp][1] = 0
                    
    def maximum_usage():
        """
        return the maximum usage
        Parameters
        ----------
        None
        
        Returns
        -------
        int
        """
        maximum = 1
        for sp in switchParam:
            if switchParam[sp][0] > maximum :
                maximum = switchParam[sp][0]
        return maximum
        
    def checkifZero_avalancheCosts():
        """
        checks if there are some types of switches
        which got their avalancheCost reduced to zero
        and returns a boolean stating if yes or no there
        is at least one such case, and a list containing
        all the types wich cost has been reduced to zero
        
        Parameters
        ----------
        None
        
        Returns
        -------
        (boolean, list[str])
        
        """
        ret=[]
        zero = False
        for typee in switchParam :
            if avalancheCost(switchParam[typee][0],switchParam[typee][1],switchParam[typee][2]) == 0 :
                ret.append(typee)
                zero = True
        return (zero,ret) 
        
    def checkifThreshold_avalancheCosts():
        """
        checks if there are some types of switches
        which got their avalancheCost reduced to a
        certain level, and returns a list containing
        all the types witch are below a certain level 
        
        Parameters
        ----------
        None
        
        Returns
        -------
        List[str]
        
        """
        maximum = 0
        ret=[]
        for typee in switchParam :
            if switchParam[typee][0] > maximum :
                maximum = switchParam[typee][0]

        threshold = maximum / teta
        for typee in switchParam :
            if switchParam[typee][0] >= threshold : # the >= here is not precise( it compares really small doubles)
                ret.append(typee)
        return ret 
    
    def removeTypesFromGraph(switchTypes):
        """
        Removes all switches, which are of a type
        listed in switchTypes, from the graph.
        Note : it deletes edge_splitters from the graph
               but it only puts multiplexers in a new state
               "multiplexer_noUsage" where their avalanche cost
               is no longer used.
        
        Parameters
        ----------
        switchTypes : list[str]
                list of switch types to remove
                
        Returns
        -------
        None
        
        """
        list_to_remove=[]
        removed_types=[]
        for node, typee in graph.nodes.data('TypeOfSwitch') :   
            if (typee in switchTypes) :
                list_to_remove.append(node)
                if not( typee in removed_types) :
                    del switchParam[typee]
                    del alreadyUpdatedUsage[typee]
                    removed_types.append(typee)
                
        for node in list_to_remove: 
            node_type = graph.nodes[node].get('TypeOfNode')
            if node_type == 'multiplexer': 
                graph.nodes[node]['TypeOfNode'] ='multiplexer_noUsage'
            elif node_type == 'edge_splitter' :
                u =''
                v =''
                for n in  graph.successors(node) :
                    v = n
                for n in  graph.predecessors(node) :
                    u = n
                graph.add_edge(u,v)
                graph.remove_node(node)
            
    def is_switchUsageStillPositive() :
        """
        True if there is some switch type
        which usage is positive. False otherwise
        
        Parameters
        ----------
        None
                
        Returns
        -------
        boolean
        
        """
        for sp in switchParam :
            if switchParam[sp][0] > 0 :
                return True
        return False
    
    # main loop -----------------------------------------------------------------------------------------------
    # this set will be used in the first iteration only to determine how many muxs are actually 
    # used per lvl. And consequently, the others will be delted to increase the speed of onvergence
    used_muxs = set() 
            
    while ( (iteration==0)  or is_switchUsageStillPositive() ):
        
        reset()
        congestion_multiplier = 0.5
        iteration = iteration + 1
        print('iteration: '  , iteration)
        i=0
        shared_resources=True
        while ( shared_resources ) :
            i+=1
            print('i: ',i)
            shared_resources = False
            used_muxs = set()
            for (source, sinks) in nets.items() : 
                path = set()          
                for sink in sinks :
                    u = nx.dijkstra_path(graph,source,sink, weight)
                    path = set(u) | path
                    setNodesToZero(path)
                    
                if( iteration == 1) :
                    used_muxs = used_muxs | path
                    
                for node in path:
                    graph.nodes[node]['isZero'] = False  #putting back the nodes to their orignal state, before next net
                update_costs(path) 
                
            if (iteration == 1 and i== 1) :
                ap = s /(maximum_usage()*(iter_to_zero + 1))
                ah = ap
                print('ap: ',ap)

            shared_resources = updateHistoryCosts_andReset(i)
            congestion_multiplier = congestion_multiplier * congestion_updater  
          
        # check how many multiplexers were used, and delete the others
        if (iteration == 1 and opt):
            maxi= 0
            used = set()
            for node in used_muxs :
                if (graph.nodes[node].get('TypeOfNode') == 'multiplexer') :
                    i = node.find('x')
                    num = int(node[i+1:])
                    if not(num in used) : 
                        maxi+=1
                        used.add(num)
            
            if maxi < num_muxs - 5 : 
                print("used muxs: ",maxi)
                multiplexer_perlvl ={}
                for n in range(numLvls) :
                    multiplexer_perlvl[n+1] = maxi + 4
                    
                outputs = make_graph(num_luts, clbNumber, numLvls, multiplexer_perlvl, maxjump)
                return routing(outputs[0], clbNumber, nets, outputs[1], num_luts, maxi+3, numLvls, maxjump)
        # end of the part that chacks how many muxs are actually used
        
        list_selected = checkifZero_avalancheCosts() 
        list_selectedBis =[]
        if list_selected[0] :
            selected_switches = selected_switches + list_selected[1]
            removeTypesFromGraph(list_selected[1])
            list_selectedBis = list_selected[1]
        else :
            list_selected = checkifThreshold_avalancheCosts()
            selected_switches = selected_switches + list_selected
            removeTypesFromGraph(list_selected)
            list_selectedBis= list_selected
        #print(selected_switches)
       
            
            
    return selected_switches   
    

In [11]:
def drawGraph(num_luts, num_ports, num_lvls, selected_switches, highlight=[]) :
    """
    
    
    """
    
    width = (150 * num_luts)/8
    height= 50
    first_distance = 15
    muxs = 0
    edges =[]
    node_fanInOut = {}
    src=[]
    dst=[]
    
    for e in selected_switches :
        if e.count('->') == 0 :
            i = e.find('x')
            i = int(e[i+1:])
            if i > muxs :
                muxs = i
        else :
            edges.append(e)
            
    pas_x = width/num_luts
    x = -(width/2)
    y = 25
    dotgraph = dot.Graph(graph_name='G', graph_type= 'digraph', suppress_disconnected = False)
    for i in range(num_luts) :
        s = 'lut' + str(i)
        dotgraph.add_node(dot.Node(name=(s+'_output'), pos = str(x)+","+str(y)+"!")) 
        dotgraph.add_node(dot.Node(name=(s+'_input'),pos = str(x)+","+str(-y)+"!") )
        node_fanInOut[s+'_output'] = [0,0]
        #node_fanInOut[s+'_input'] = [0,0]
        
        x2 = x- (pas_x/2 -1)
        pas_x2 = (pas_x - 2)/num_ports
        for p in range(num_ports) :
            dotgraph.add_node( dot.Node( name= (s+'_port'+str(p)), pos = str(x2)+","+str(-y+5)+"!" ) ) 
            #dotgraph.add_edge( dot.Edge(  src=(s+'_port'+str(p)), dst=(s+'_input')  )  )
            src.append([(s+'_port'+str(p)),(s+'_port'+str(p)) in highlight])
            dst.append([(s+'_input'),(s+'_input') in highlight])
            node_fanInOut[s+'_port'+str(p)] = [0,1]
            x2 = x2 + pas_x2
        x= x + pas_x 
        
        
    x = -(width/2)-5
    y = y-first_distance
    
    for lvl in range(num_lvls) :
        for m in range(muxs+1) :
            dotgraph.add_node( dot.Node( name=('level'+str(lvl+1)+'_mux'+str(m)), pos = str(x) +","+str(y)+"!" ) )
            node_fanInOut['level'+str(lvl+1)+'_mux'+str(m)] = [0,0]
            x = x + (width + 10)/muxs
        x = -(width/2)-5
        y = y - ((2*(height/2)-2*first_distance)/num_lvls)
    for e in edges :
        couple = e.rsplit(' -> ')
        #dotgraph.add_edge(  dot.Edge( src=couple[0], dst=couple[1] )  )
        src.append([couple[0],(couple[0] in highlight)])
        dst.append([couple[1],(couple[1] in highlight)])
        node_fanInOut[couple[0]][1] += 1
        node_fanInOut[couple[1]][0] += 1
    
    for node in node_fanInOut :
        instance = (dotgraph.get_node(node))[0]
        instance.set_name(node + " - "+ str(node_fanInOut[node][0])+":"+str(node_fanInOut[node][1])+" -" )
        for i in range(len(src)):
            if src[i][0] == node :
                src[i][0] = node + " - "+ str(node_fanInOut[node][0])+":"+str(node_fanInOut[node][1])+" -"
            if dst[i][0] == node :
                dst[i][0] = node + " - "+ str(node_fanInOut[node][0])+":"+str(node_fanInOut[node][1])+" -"

    for i in range(len(src)):
        if (src[i][1] or dst[i][1]) :
            dotgraph.add_edge( dot.Edge(src= src[i][0], dst = dst[i][0], arrowsize = 3, color = "red") )
        else :    
            dotgraph.add_edge( dot.Edge(src= src[i][0], dst = dst[i][0]) )
        
    return dotgraph.to_string()



